In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Dropout
from tensorflow.keras.callbacks import EarlyStopping

plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams.update({'font.size':20})
plt.rcParams["figure.figsize"] = (12,10)

In [2]:
str_files = ''
#Carrega os dados do arquivo .csv
df = pd.read_csv(str_files + "cetesb_concatenado06semoutliers.csv",encoding='utf-8',sep=',')
trainingd = df
df

,ph,coliformes,dbo,fosforo,od,solido,temperatura,turbidez
0,6.415254,1.492010e+06,19.932203,0.725203,1.288983,291.305085,21.338983,62.362712
1,6.404952,1.587547e+06,19.122792,0.706433,1.355509,290.850631,21.371426,62.670960
2,6.393545,1.693320e+06,18.226659,0.685652,1.429162,290.347485,21.407346,63.012236
3,6.382507,1.795681e+06,17.359433,0.665542,1.500439,289.860570,21.442106,63.342502
4,6.371100,1.901455e+06,16.463299,0.644761,1.574092,289.357425,21.478026,63.683777
...,...,...,...,...,...,...,...,...
498,7.340000,8.115310e+05,41.025974,0.318414,0.420000,344.767399,21.761966,32.000000
499,7.370000,9.301910e+05,26.129870,0.546829,0.720000,336.690934,25.439754,34.939430
500,6.827000,1.045023e+06,11.714286,0.767875,2.256250,328.875000,24.075000,45.714286
501,6.920000,1.045023e+06,11.714286,0.767875,2.256250,328.875000,21.140164,45.714286


In [3]:
def pegar_dados_coluna_predita_train_test(trainingd,percent,index_coluna):
    data = trainingd.iloc[:,:].values
    train = trainingd.iloc[0:int(len(data)*percent),:].values  
    train_previsao = trainingd.iloc[0:int(len(data)*percent),index_coluna:index_coluna+1].values
    test = trainingd.iloc[len(train):,index_coluna:index_coluna+1].values
    '''
    print('Observações.:',len(data))
    print('Treino.:',len(train))
    print('Test.:',len(test))
    '''
    return train,train_previsao, test

#Normalização dos dados: Normaliza os dados dentro um intervalo (0 a 1).
def normalizacao(train,test):
    sc = MinMaxScaler()
    testd = test
    train = sc.fit_transform(train)
    test = sc.fit_transform(test)    
    return train,test,testd

def prepara_dados_treino(trainingd,train,test,lags):
    #Prepara os dados de teste
    entradas = trainingd[:len(trainingd) - len(test) - lags].values
    entradas = sc.fit_transform(entradas)  
    
        
    test_X = []
    for i in range(lags, lags+len(train)):
        test_X.append(entradas[i-lags:i, 0:8])
    test_X = np.array(test_X)
        
    return test_X
    

#Prepara o conjunto de dados em X e y, considerando a janela de visualização (lags).
#cy = coluna que será predita
def prepara_dados(dados,lags,cy):
    X = []
    y = []
    for i in range(lags, len(dados)):
        X.append(dados[i-lags:i,0:8])
        y.append(dados[i, cy])
       
    return np.array(X), np.array(y)

#Calcula o MAPE
#Define função para calcular o MAPE
#def mape(y_pred,y_true):
 #   return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mape(y_pred,y_true):
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    '''  
    mape_sum = 0
    for real,prediction in zip(y_true,y_pred):
        mape_sum += (abs((real - prediction))/real)
        
        print(real)
        mape = mape_sum/len(real)
    '''
    return mape

def rmse(y_pred,y_true):
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    return rmse

def r2(y_pred,y_true):
    rscore = r2_score(y_true,y_pred)
    return rscore

def rquadrado(y_pred,y_true):
    #Soma Total dos Quadrados (STQ): mostra a variação de y em torno da própria média. 
    #É o somatório das diferenças entre o valor alvo real e sua média elevado ao quadrado.
    y_traco = np.mean(y_true)
    
    print('media y_true:', y_traco)
        
    stq = 0
    
    for s in y_true:
        a = s - y_traco
        st = a * a
        stq = stq + st
    
    print('Soma Total dos Quadrados (STQ):', stq[0])
    
    #Soma dos Quadrados dos Resíduos (SQU): variação de Y que não é explicada pelo modelo elaborado. 
    #É o somatório das diferenças entre o valor predito e o valor real elevados ao quadrado.
    squ = 0
    
    y_traco = np.mean(y_pred)
    print('media y_pred:', y_traco)
    
    for n in range(len(y_true)):
        a = y_true[n] - y_pred[n]
        st = a * a
        squ = squ + st
        
    print('Soma dos Quadrados dos Resíduos (SQU):',squ[0])
    print('\n')
    print('Fórmula do R²')
    print('\n')
    print('sqr = stq - squ')
    sqr = stq[0] - squ[0]
    print('R² = sqr/stq')
    sqr = sqr/stq[0]
    print('\n')
    return sqr

def correlacao_determinacao(dtframe,tipo):
    
    if (tipo == 0): #Treino
        resultado = dtframe.corr().previsao_treino.values[1] ** 2 
    else: #teste
        resultado = dtframe.corr().previsao_teste.values[1] ** 2
    
    return resultado

def ajusta_array(array):
    lista = []
    
    for i in range(len(array)):
        lista.append(array[i][0])
        
    #print('ajusta array:',lista)
        
    return lista

def ajusta_lista(array):
    lista = []
    
    for i in range(len(array)):
        lista.append(array[i][0])
        
    #print('ajusta array:',lista)
        
    return lista

In [4]:

df_resultado_medio_g = pd.DataFrame()

index_coluna = 0

parametro = []

#média
media_parametro_mape_treino = []
media_parametro_rmse_treino = []
media_parametro_r_treino    = []
   
media_parametro_mape_teste  = []
media_parametro_rmse_teste  = []
media_parametro_r_teste     = []
   
#desvio_padrão
   
std_parametro_mape_treino = []
std_parametro_rmse_treino = []
std_parametro_r_treino    = []
   
std_parametro_mape_teste  = []
std_parametro_rmse_teste  = []
std_parametro_r_teste     = []

for p in ('ph','coliformes', 'dbo','fosforo','od','solido','temperatura','turbidez'):
   
    print('Agora é a vez do parâmetro:', p)
      
    df_resultados_treino = pd.DataFrame()
    df_resultados_teste  = pd.DataFrame()
    df_resultados        = pd.DataFrame()
   
    media_lag_mape_treino = []
    media_lag_rmse_treino = []
    media_lag_r_treino    = []
       
    media_lag_mape_teste = []
    media_lag_rmse_teste = []
    media_lag_r_teste    = []

    for l in range(1, 16):
    
        lags = l
    
        #seleciona os dados
    
        train,train_previsao, test = pegar_dados_coluna_predita_train_test(trainingd,0.70,index_coluna) 
    
        #normalização dos dados
        train,test,testd = normalizacao(train,test)
        normalizador_previsao = MinMaxScaler()
        sc = MinMaxScaler()
        normalizador_previsao.fit_transform(train_previsao)
    
        #Prepara os dados de treinamento
        train_X, train_y = prepara_dados(train, lags,index_coluna)
    
        #Prepara os dados de teste
        entradas = trainingd[len(trainingd) - len(test) - lags:].values
        entradas = sc.fit_transform(entradas)   
    
        test_X = []
        for i in range(lags, lags+len(test)):
            test_X.append(entradas[i-lags:i, 0:8])
        test_X = np.array(test_X)
    
        media_simulador_mape_treino = []
        media_simulador_rmse_treino = []
        media_simulador_r_treino    = []
            
        media_simulador_mape_teste  = []
        media_simulador_rmse_teste  = []
        media_simulador_r_teste     = []
        
        for r in range(0,6):

            model = Sequential()
            model.add(LSTM(units = 10, input_shape = (train_X.shape[1], 8)))
            model.add(Dense(21, activation = 'relu'))
            model.add(Dropout(0.2))
            model.add(Dense(1, activation = 'sigmoid'))
            model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])


            es = EarlyStopping(monitor='val_loss', patience = 3, verbose=0)


            #Treina o modelo
            history = model.fit(train_X, train_y, validation_data=(test_X, test), batch_size = 32, epochs = 2000, 
                        callbacks=[es], verbose=0)
    
    
            #Dados de teste
            previsoes = model.predict(test_X)
            #previsoes = previsoes.reshape(-1, 1)
            previsoes = normalizador_previsao.inverse_transform(previsoes)
            '''
            print('Teste - Gráficos com lag', l)
            #Plotagem do gráfico
            plt.plot(testd,color='red',label = 'Observado')
            plt.plot(previsoes,color='blue',label = 'Previsoes')
            plt.xlabel('Tempo')
            plt.ylabel('Valor pH')
            plt.legend()
            plt.show()
            '''
    
            #Dados de treino
            previsoes_treino = model.predict(train_X)
            previsoes_treino = previsoes_treino.reshape(-1, 1)
            previsoes_treino = normalizador_previsao.inverse_transform(previsoes_treino)
    
            treino = train_previsao[lags: len(previsoes_treino) + lags, :]
            observado_test = testd
    
            observado_treino = train_previsao  
            '''
            print('Treinamento - Gráficos com lag', l)
            #Plotagem do gráfico
            plt.plot(train_previsao,color='red',label = 'Observado')
            plt.plot(previsoes_treino,color='blue',label = 'Previsoes')
            plt.xlabel('Tempo')
            plt.ylabel('Valor pH')
            plt.legend()
            plt.show()
            '''        
            treino         = ajusta_array(treino)
            observado_test = ajusta_array(observado_test)
               
            #Calculo do erro da previsão MAPE, RMSE e R²
    
            observado_treino_d = observado_treino
            previsoes_treino_d = previsoes_treino
            previsoes_d        = previsoes
          
            mape_treino_d        = round(mape(previsoes_treino,treino),2)
            rmse_treino_d        = round(rmse(previsoes_treino,treino),2)
            #r_treino_d           = round(r2(previsoes_treino,treino),4)
        
            mape_teste_d        = round(mape(previsoes,observado_test),2)
            rmse_teste_d        = round(rmse(previsoes,observado_test),2)
            #r_teste_d           = round(r2(previsoes,observado_test),4) 
       
            df_corr_determinacao_treino = pd.DataFrame()
            df_corr_determinacao_teste  = pd.DataFrame()
    
            #Calcula o coeficiente de determinação
            dict = {'previsao_treino': ajusta_lista(previsoes_treino), 'treino': treino} 
    
            df_treino = pd.DataFrame(dict)
            dframes_treino = [df_corr_determinacao_treino,df_treino]
            df_corr_determinacao_treino = pd.concat(dframes_treino)
    
            dict = {'previsao_teste': ajusta_lista(previsoes), 'teste': observado_test} 
    
            df_teste = pd.DataFrame(dict)
            dframes_teste = [df_corr_determinacao_teste,df_teste]
            df_corr_determinacao_teste = pd.concat(dframes_teste)
        
            r_treino_d = []
            r_teste_d  = []
    
            r_treino_d.append(round(correlacao_determinacao(df_corr_determinacao_treino,0),2))
            r_teste_d.append(round(correlacao_determinacao(df_corr_determinacao_teste,1),2))
            
            media_simulador_mape_treino.append(np.mean(mape_treino_d))
            media_simulador_rmse_treino.append(np.mean(rmse_treino_d))
            media_simulador_r_treino.append(np.mean(r_treino_d))
            
            media_simulador_mape_teste.append(np.mean(mape_teste_d))
            media_simulador_rmse_teste.append(np.mean(rmse_teste_d))
            media_simulador_r_teste.append(np.mean(r_teste_d))
            
            
    
        media_lag_mape_treino.append(np.mean(media_simulador_mape_treino))
        media_lag_rmse_treino.append(np.mean(media_simulador_rmse_treino))
        media_lag_r_treino.append(np.mean(media_simulador_r_treino))
        
        media_lag_mape_teste.append(np.mean(media_simulador_mape_teste))
        media_lag_rmse_teste.append(np.mean(media_simulador_rmse_teste))
        media_lag_r_teste.append(np.mean(media_simulador_r_teste))
        
    index_coluna = index_coluna + 1            
    #média
    media_parametro_mape_treino.append(np.mean(media_lag_mape_treino))
    media_parametro_rmse_treino.append(np.mean(media_lag_rmse_treino))
    media_parametro_r_treino.append(np.mean(media_lag_r_treino))
    
    media_parametro_mape_teste.append(np.mean(media_lag_mape_teste))
    media_parametro_rmse_teste.append(np.mean(media_lag_rmse_teste))
    media_parametro_r_teste.append(np.mean(media_lag_r_teste))
        
    #desvio_padrão
    
    std_parametro_mape_treino.append(np.std(media_lag_mape_treino))
    std_parametro_rmse_treino.append(np.std(media_lag_rmse_treino))
    std_parametro_r_treino.append(np.std(media_lag_r_treino))
    
    std_parametro_mape_teste.append(np.std(media_lag_mape_teste))
    std_parametro_rmse_teste.append(np.std(media_lag_rmse_teste))
    std_parametro_r_teste.append(np.std(media_lag_r_teste))
        
    parametro.append(p)
    
dict = {'parametro': parametro,
        'mape_treino':  media_parametro_mape_treino, 'std_mape_treino':  std_parametro_mape_treino, 
        'rmse_treino':  media_parametro_rmse_treino, 'std_rmse_treino':  std_parametro_rmse_treino,
        'r_quad_treino':media_parametro_r_treino,    'std_r_quad_treino':std_parametro_r_treino,
        'mape_teste':   media_parametro_mape_teste,  'std_mape_teste':   std_parametro_mape_teste, 
        'rmse_teste':   media_parametro_rmse_teste,  'std_rmse_teste':   std_parametro_rmse_teste,
        'r_quad_teste': media_parametro_r_teste,     'std_r_quad_teste': std_parametro_r_teste}
   
df_resultado_final = pd.DataFrame(dict)

dframes = [df_resultado_medio_g,df_resultado_final]
df_resultado_medio_g = pd.concat(dframes)

df_resultado_medio_g.round(2)

Agora é a vez do parâmetro: ph
Agora é a vez do parâmetro: coliformes
Agora é a vez do parâmetro: dbo
Agora é a vez do parâmetro: fosforo
Agora é a vez do parâmetro: od
Agora é a vez do parâmetro: solido
Agora é a vez do parâmetro: temperatura
Agora é a vez do parâmetro: turbidez


,parametro,mape_treino,std_mape_treino,rmse_treino,std_rmse_treino,r_quad_treino,std_r_quad_treino,mape_teste,std_mape_teste,rmse_teste,std_rmse_teste,r_quad_teste,std_r_quad_teste
0,ph,3.84,0.06,0.28,0.01,0.38,0.03,4.64,0.11,0.40,0.01,0.07,0.02
1,coliformes,153587.05,2194.47,1014510.96,12616.08,0.21,0.02,1752581.24,168806.54,986152.78,119592.75,0.01,0.01
2,dbo,100.02,1.17,13.34,0.06,0.05,0.01,40.18,0.24,14.53,0.23,0.02,0.01
3,fosforo,116.87,1.52,0.55,0.00,0.05,0.01,101.99,1.10,0.48,0.00,0.03,0.01
4,od,168.78,2.76,0.96,0.02,0.11,0.02,74.86,1.54,0.75,0.01,0.07,0.03
5,solido,42.60,0.29,82.60,2.20,0.26,0.03,18.56,0.41,72.54,2.00,0.05,0.02
6,temperatura,6.05,0.06,1.58,0.02,0.09,0.02,6.67,0.08,1.74,0.02,0.02,0.02
7,turbidez,61.14,4.09,14.05,0.59,0.05,0.01,34.86,1.94,16.36,0.99,0.00,0.00
